In [13]:
!pip install tensorflow
!pip install keras

     -------------------------------------- 284.1/284.1 MB 1.9 MB/s eta 0:00:00
     ------------------------------------- 440.7/440.7 kB 13.9 MB/s eta 0:00:00
     ---------------------------------------- 3.7/3.7 MB 16.7 MB/s eta 0:00:00
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp310-cp310-win_amd64.whl (1.5 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached ml_dtypes-0.2.0-cp310-cp310-win_amd64.whl (938 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
     ---------------------------------------- 1.7/1.7 MB 15.6 MB/s eta 0:00:00
     -------------------------------------- 130.2/130.2 kB 7.5 MB/s eta 0:00:00
     ---------------------------------------- 5.5/5.5 MB 14.1 MB/s eta 0:00:00
  Using cached termcolor-2.3.0-py3-none-any.whl (6.9 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
     --------------------------------------- 24.4/24.4 MB 14.5 MB/s eta 0

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from nltk import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Dropout, LSTM, Bidirectional, Input, Activation
from gensim.models import Word2Vec
import gensim.downloader as api 

In [10]:
d = pd.read_csv('train.csv')

In [11]:
d.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [16]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1306122 entries, 0 to 1306121
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   qid            1306122 non-null  object
 1   question_text  1306122 non-null  object
 2   target         1306122 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 29.9+ MB


In [17]:
keras_token = Tokenizer(num_words = 50000, char_level=False, split = ' ')
keras_token.fit_on_texts(d['question_text'])

In [32]:
len(keras_token.word_index)

222161

In [18]:
seq = keras_token.texts_to_sequences(d['question_text'])
max_len = 50
seq_matrix = sequence.pad_sequences(seq, maxlen=max_len)
seq_matrix

array([[    0,     0,     0, ...,     6,     1,  8333],
       [    0,     0,     0, ...,    10,    44,  1846],
       [    0,     0,     0, ...,   374,   451,  5546],
       ...,
       [    0,     0,     0, ...,  7760, 11663,  2984],
       [    0,     0,     0, ...,    43,  3384,   422],
       [    0,     0,     0, ...,    10,     4, 21972]])

In [19]:
word2vec = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [33]:
vocabulary_size

3000000

In [21]:
words = list(word2vec.index_to_key)
vocabulary_size = len(words)
embedding_dim = len(word2vec['house'])

embedding_matrix = np.zeros((vocabulary_size, embedding_dim))
for word, i in keras_token.word_index.items():
    if word in word2vec:
        embedding_matrix[i] = word2vec[word]

In [22]:
y = d['target']
y = np.array(y)
X = seq_matrix

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [24]:
model = Sequential()
model.add(Embedding(input_dim=vocabulary_size, 
                    output_dim=embedding_dim, input_length=50, 
                    weights=[embedding_matrix], trainable=False))
model.add(GlobalAveragePooling1D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 300)           900000000 
                                                                 
 global_average_pooling1d (  (None, 300)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 256)               77056     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                        

C:\Users\hp\anaconda3\lib\site-packages\keras\src\utils\layer_utils.py:146: RuntimeWarning: overflow encountered in long_scalars
  total_memory_size += weight_shape * per_param_size


In [25]:
model.compile(loss='binary_crossentropy',optimizer='adam' ,metrics=['accuracy'])

In [26]:
num_epochs = 30
early_stop = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(X_train, y_train, epochs=num_epochs, 
                    validation_data=(X_validation, y_validation),callbacks =[early_stop], verbose=1)

Epoch 1/30
22858/22858 [==============================] - 242s 10ms/step - loss: 0.1343 - accuracy: 0.9489 - val_loss: 0.1288 - val_accuracy: 0.9507
Epoch 2/30
22858/22858 [==============================] - 244s 11ms/step - loss: 0.1256 - accuracy: 0.9510 - val_loss: 0.1228 - val_accuracy: 0.9518
Epoch 3/30
22858/22858 [==============================] - 248s 11ms/step - loss: 0.1229 - accuracy: 0.9520 - val_loss: 0.1240 - val_accuracy: 0.9524
Epoch 4/30
22858/22858 [==============================] - 220s 10ms/step - loss: 0.1211 - accuracy: 0.9527 - val_loss: 0.1215 - val_accuracy: 0.9526
Epoch 5/30
22858/22858 [==============================] - 251s 11ms/step - loss: 0.1199 - accuracy: 0.9531 - val_loss: 0.1191 - val_accuracy: 0.9528
Epoch 6/30
22858/22858 [==============================] - 248s 11ms/step - loss: 0.1186 - accuracy: 0.9536 - val_loss: 0.1186 - val_accuracy: 0.9529
Epoch 7/30
22858/22858 [==============================] - 232s 10ms/step - loss: 0.1175 - accuracy: 0.9540

In [27]:
pred = model.predict(X_test)
threshold = 0.5
binary_predictions = (pred >= threshold).astype(int)

12245/12245 [==============================] - 55s 5ms/step


In [38]:
pred = pd.DataFrame(pred)
pred.describe()

,0
count,391837.000000
mean,0.062242
std,0.158490
min,0.000000
25%,0.000074
50%,0.001408
75%,0.017424
max,0.977440


In [28]:
from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(y_test, binary_predictions)

print("Confusion Matrix:")
print(confusion)

true_positives = confusion[1, 1]
false_positives = confusion[0, 1]

precision = true_positives / (true_positives + false_positives)

print("Precision:", precision)

Confusion Matrix:
[[362095   5878]
 [ 11516  12348]]
Precision: 0.6774936903324921
